<h1>Battle of Neighbourhoods</h1>

<h2>1. Introduction/Business Problem</h2>

The objective of this capstone project is to analyse and select the best locations in the city of Washington, USA to open a new shopping mall. Using data science methodology and machine
learning techniques like clustering, this project aims to provide solutions to answer the business
question: In the city of Washington,USA if a property developer is looking to open a new
shopping mall, where would you recommend that they open it?

<h2>2. Data Section</h2>

To solve the problem, we will need the following data:
* List of neighbourhoods in Washington. This defines the scope of this project which is
  confined to the city of Washington, the capital city of the country of USA.
* Latitude and longitude coordinates of those neighbourhoods. This is required in order to
  plot the map and also to get the venue data.
* Venue data, particularly data related to shopping malls. We will use this data to perform
  clustering on the neighbourhoods
 
we can use the "**List of cities and towns in Washington**" data to solve our problem. Data can be found at  [https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Washington](http://)
    

<h3>2.1 Data preprocess</h3>

In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize
import folium # plotting library

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Washington'
tables = pd.read_html(url,skiprows=0)[0]
df1=pd.DataFrame(tables)
df1.columns = df1.columns.map(''.join)
df1.head(50)


,NameName,Type[3]Type[3],County[4]County[4],Population2018 estimate[6],Population2010 Census[5],Land area[5]sq mi,Land area[5]km2
0,Aberdeen,First,Grays Harbor,16654,16896,10.65,27.6
1,Airway Heights,Code,Spokane,9200,6114,5.12,13.3
2,Algona,Code,King,3223,3014,1.30,3.4
3,Anacortes,Code,Skagit,17283,15778,11.76,30.5
4,Arlington,Code,Snohomish,19803,17926,9.30,24.1
5,Asotin,Code,Asotin,1295,1251,1.05,2.7
6,Auburn,Code,KingPierce[Note 1],81905,70180,29.62,76.7
7,Bainbridge Island,Code,Kitsap,24846,23025,27.61,71.5
8,Battle Ground,Code,Clark,20949,17571,7.86,20.4
9,Bellevue,Code,King,147599,122363,33.48,86.7


In [3]:
df1=df1[['NameName','County[4]County[4]','Population2010 Census[5]','Land area[5]km2']]
df1.rename(columns={'County[4]County[4]':'County','NameName':'Name','Population2010 Census[5]':'Population','Land area[5]km2':'Land Area(km2)'}, inplace=True)
for i in range(df1.shape[0]):
    if df1['County'][i][-8:] =='[Note 1]':
        df1['County'][i]="King"
df1[0:10]

C:\Users\NisanthanSelvaratnam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Name,County,Population,Land Area(km2)
0,Aberdeen,Grays Harbor,16896,27.6
1,Airway Heights,Spokane,6114,13.3
2,Algona,King,3014,3.4
3,Anacortes,Skagit,15778,30.5
4,Arlington,Snohomish,17926,24.1
5,Asotin,Asotin,1251,2.7
6,Auburn,King,70180,76.7
7,Bainbridge Island,Kitsap,23025,71.5
8,Battle Ground,Clark,17571,20.4
9,Bellevue,King,122363,86.7


In [4]:
from geopy.geocoders import Nominatim


def getCoordinates(address,county):
    geolocator = Nominatim(timeout=3, user_agent='lavan')
    address = address+", "+county
    location = geolocator.geocode(address,country_codes = ["US"])
    return location.latitude, location.longitude

In [ ]:
df1['Latitude'] = ""
df1['Longitude']=""
for i in range(df1.shape[0]):
    df1['Latitude'][i] = getCoordinates(df1['Name'][i],df1['County'][i])[0]
    df1['Longitude'][i] = getCoordinates(df1['Name'][i],df1['County'][i])[1]
df1.head()

C:\Users\NisanthanSelvaratnam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\NisanthanSelvaratnam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
df['Latitude']=""
df['Longtitude']=""
for i in range(df.shape[0]):
    try:
        df['Latitude'][i] = getCoordinates(df['Name'][i])[0]
        df['Longtitude'][i] = getCoordinates(df['Name'][i])[1]
    except:
        pass
df.head()

    

C:\Users\NisanthanSelvaratnam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\NisanthanSelvaratnam\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name,County,Population,Land Area(km2),Latitude,Longtitude
0,Aberdeen,Grays Harbor,16896,27.6,57.1482,-2.09281
1,Airway Heights,Spokane,6114,13.3,47.6446,-117.593
2,Algona,King,3014,3.4,43.07,-94.233
3,Anacortes,Skagit,15778,30.5,48.502,-122.624
4,Arlington,Snohomish,17926,24.1,32.7019,-97.1056
